## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Описание данных
 - 
children — количество детей в семье
 - 
days_employed — общий трудовой стаж в днях
 - 
dob_years — возраст клиента в годах
 - 
education — уровень образования клиента
 - 
education_id — идентификатор уровня образования
 - 
family_status — семейное положение
 - 
family_status_id — идентификатор семейного положения
 - 
gender — пол клиента
 - 
income_type — тип занятости
 - 
debt — имел ли задолженность по возврату кредитов
 - 
total_income — ежемесячный доход
 - 
purpose — цель получения кредита



План работы:
 
Шаг 1. Открыть таблицу и изучить общую информацию о данных
 
Шаг 2. Предобработка данных
 
Шаг 3. Ответить на вопросы
 
Шаг 4. Написать общий вывод

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [4]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [11]:
users

,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff
0,1000,52,NaN,Краснодар,Рафаил,Верещагин,2018-05-25,ultra
1,1001,41,NaN,Москва,Иван,Ежов,2018-11-01,smart
2,1002,59,NaN,Стерлитамак,Евгений,Абрамович,2018-06-17,smart
3,1003,23,NaN,Москва,Белла,Белякова,2018-08-17,ultra
4,1004,68,NaN,Новокузнецк,Татьяна,Авдеенко,2018-05-14,ultra
...,...,...,...,...,...,...,...,...
495,1495,65,NaN,Иркутск,Авксентий,Фокин,2018-08-28,ultra
496,1496,36,NaN,Вологда,Трифон,Блохин,2018-01-27,smart
497,1497,32,NaN,Челябинск,Каролина,Агеева,2018-10-09,smart
498,1498,68,2018-10-25,Владикавказ,Всеволод,Акимчин,2018-07-19,smart


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Отрицательные значения в столбце "days_employed".
Разный регистр в столбце "education"

### Шаг 2. Предобработка данных

Обработка пропусков

In [3]:
df.isnull().sum() #Обнаружены пропуски в days_employed и total_income

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
#Замена явно-ошибочных данных
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)
df['gender'] = df['gender'].replace('XNA', 'M')
df['income_type'].value_counts()
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [5]:
#Заполнение пропусков средним значением по столбцу для каждого типа занятости
df['days_employed'] = df.groupby('income_type')['days_employed'].transform(lambda x: x.fillna(x.mean()*df['dob_years']*365))
df['total_income'] = df.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.mean()))
df.isnull().sum() #Проверка устранения пропусков

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

 Вероятно, что пропуски данных в days_employed и total_income получились в результате того, что заемщик не имел ни опыта работы, ни официального заработка. Устранены пропуски в столбцах "days_employed" и "total_income".

### Замена типа данных

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [7]:
#Изменение типа данных float на int для большей точности данных с помощью метода astype
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Для большей точности столбцам "days_employed" и "total_income" присвоено целочисленное значение

### Обработка дубликатов

In [8]:
#Проверка наличия дубликатов
df.duplicated().sum()
df[df.duplicated(keep=False)].sort_values(['total_income', 'days_employed'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
17787,0,7194218812,54,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции с жильем
21415,0,7194218812,54,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции с жильем
3344,0,7460671361,56,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции со своей недвижимостью
9627,0,7460671361,56,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции со своей недвижимостью
1681,0,7593897635,57,среднее,1,гражданский брак,1,F,пенсионер,0,137127,на проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...
17774,1,-30828256,40,среднее,1,гражданский брак,1,F,компаньон,0,202417,строительство жилой недвижимости
9374,0,-29286843,38,высшее,0,гражданский брак,1,F,компаньон,0,202417,на проведение свадьбы
19387,0,-29286843,38,высшее,0,гражданский брак,1,F,компаньон,0,202417,на проведение свадьбы
8490,1,-23891898,31,среднее,1,женат / замужем,0,F,компаньон,0,202417,покупка жилья


In [9]:
df['education'] = df['education'].str.lower() 
#Устранение дубликатов с восстановлением индексов 
df = df.drop_duplicates().reset_index(drop = True)
df.duplicated().sum()

0

### Вывод

Избавились от грубых дубликатов, которые могли появиться из-за ошибки ввода данных, так как дубликаты все находятся на разных строках (то есть не автоматическое задвоение)

### Лемматизация

In [10]:
#Исследование уникальных значений в столбце purpose
unique_purposes = df['purpose'].value_counts()
unique_purposes

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [11]:
#Лемматизировала слова в столбце purpose
from pymystem3 import Mystem
m = Mystem()
df['lem_purpose'] = df['purpose'].apply(m.lemmatize)
df['lem_purpose'].value_counts()

[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            624
[покупка

In [12]:
list_lemm = ['автомобиль', 'свадьба', 'недвижимость', 'образование', 'жилье']
# Разделала цели на основные категории  
def find_purpose(df):
    for purposes in list_lemm:
        if purposes in df:
            return purposes
df['new_purpose'] = df['lem_purpose'].apply(find_purpose)
df['new_purpose'].value_counts()

недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: new_purpose, dtype: int64

### Вывод

Выделен список основных целей кредита

### Категоризация данных

In [13]:
#Выявление процента должников по кредитам по отношению к количеству детей

child_purs = df.groupby('children')['debt'].agg(['count','mean'])
child_purs.sort_values('mean')

,count,mean
children,,
5,9,0.000000
0,14091,0.075438
3,330,0.081818
1,4855,0.091658
2,2128,0.094925
4,41,0.097561


In [14]:
#Выявление процента должников по кредитам по отношению к семейному положению

fam_purs = df.groupby('family_status')['debt'].agg(['count','mean'])
fam_purs.sort_values('mean')

,count,mean
family_status,,
вдовец / вдова,959,0.065693
в разводе,1195,0.071130
женат / замужем,12339,0.075452
гражданский брак,4151,0.093471
Не женат / не замужем,2810,0.097509


In [15]:
#Выделение количество детей на более узкие категории для большей точности расчетов
def children_cat(row):
    if row['children'] == 0:
        return 'нет детей'
    elif row['children'] <= 2:
        return '1-2 ребенка'
    else:
        return 'многодетные'
    
df['children_cat'] = df.apply(children_cat, axis=1)
df['children_cat']

0        1-2 ребенка
1        1-2 ребенка
2          нет детей
3        многодетные
4          нет детей
            ...     
21449    1-2 ребенка
21450      нет детей
21451    1-2 ребенка
21452    многодетные
21453    1-2 ребенка
Name: children_cat, Length: 21454, dtype: object

In [16]:
#Выделение количества детей на более узкие категории для большей точности расчетов
def children_cat2(row):
    if row['children'] == 0:
        return 'нет детей'
    else:
        return 'есть дети'
    
df['children_cat2'] = df.apply(children_cat2, axis=1)
df['children_cat2']

0        есть дети
1        есть дети
2        нет детей
3        есть дети
4        нет детей
           ...    
21449    есть дети
21450    нет детей
21451    есть дети
21452    есть дети
21453    есть дети
Name: children_cat2, Length: 21454, dtype: object

In [17]:
#Выявление существующих статусов семейного положения
df['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [18]:
#Разбивка заемщиков по семейному положению на две крупные категории

def family_cat(row):
    if row['family_status'] == 'женат / замужем':
        return 'семейные пары'
    elif row ['family_status'] =='гражданский брак':
        return 'семейные пары'
    else:
        return 'одинокие'
    
df['family_cat'] = df.apply(family_cat, axis=1)
df['family_cat']

0        семейные пары
1        семейные пары
2        семейные пары
3        семейные пары
4        семейные пары
             ...      
21449    семейные пары
21450    семейные пары
21451    семейные пары
21452    семейные пары
21453    семейные пары
Name: family_cat, Length: 21454, dtype: object

In [19]:
#Разделение доходов заемщиков на категории, приближаясь к равному количеству заемщиков в каждой категории
df['total_income_interval'] = pd.cut(df['total_income'], [20000, 100000, 135000, 165000, 215000, 3000000])
df.groupby('total_income_interval')['debt'].agg(['count','mean']).sort_values('total_income_interval')

,count,mean
total_income_interval,,
"(20000, 100000]",4463,0.079319
"(100000, 135000]",4075,0.083681
"(135000, 165000]",4562,0.089654
"(165000, 215000]",4084,0.082762
"(215000, 3000000]",4270,0.070023


In [20]:
#Объединение доходов заемщиков на категории для более наглядной оценки (по равному количеству заемщиков) для построения более надежных выводов.
def total_income_cat(row):
    if 20000 <= row['total_income'] <= 100000:
        return 'Нижний среднего класса'
    if 100000 < row['total_income'] <= 135000:
        return 'Cредний класс'
    if 135000 < row['total_income'] <= 165000:
        return 'Выше среднего класса'
    if 165000 < row['total_income'] <= 215000:
        return 'Обеспеченные'
    if 215000 < row['total_income'] <= 3000000:
        return 'Богатые'
    
df['total_income_cat'] = df.apply(total_income_cat, axis=1)
df['total_income_cat']

0                       Богатые
1                 Cредний класс
2          Выше среднего класса
3                       Богатые
4          Выше среднего класса
                  ...          
21449                   Богатые
21450      Выше среднего класса
21451    Нижний среднего класса
21452                   Богатые
21453    Нижний среднего класса
Name: total_income_cat, Length: 21454, dtype: object

In [21]:
#Объединение целей кредита на общие категории 
def purpose_cat(row):
    if row['new_purpose'] == 'жилье':
        return 'Недвижимость'
    if row['new_purpose'] == 'недвижимость':
        return 'Недвижимость'
    if row['new_purpose'] == 'свадьба':
        return 'Свадьба'
    if row['new_purpose'] == 'образование':
        return 'Образование'
    if row['new_purpose'] == 'автомобиль':
        return 'Автомобиль'
    
df['purpose_cat'] = df.apply(purpose_cat, axis=1)
df['purpose_cat']

0        Недвижимость
1          Автомобиль
2        Недвижимость
3         Образование
4             Свадьба
             ...     
21449    Недвижимость
21450      Автомобиль
21451    Недвижимость
21452      Автомобиль
21453      Автомобиль
Name: purpose_cat, Length: 21454, dtype: object

### Вывод

Основные параметры объединены в более крупные категории для наглядности оценки платёжеспособности заемщиков. Однако, количество заемщиков в разных по разным параметрам сильно варьируется, что может влиять на общий анализ.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [22]:
child_cat_purs = df.groupby('children_cat')['debt'].agg(['count','mean'])
child_cat_purs.sort_values('mean')

,count,mean
children_cat,,
нет детей,14091,0.075438
многодетные,380,0.081579
1-2 ребенка,6983,0.092654


### Вывод

Зависимость между наличием детей и возвратом кредита в срок есть, заемщики без детей чаще выплачивают кредит в срок, чем заемщики, у которых есть дети.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
#Выявление процента должников по кредитам по отношению к семейному положению с разбивкой на категории

family_cat_purs = df.groupby('family_cat')['debt'].agg(['count','mean'])
family_cat_purs.sort_values('mean')


,count,mean
family_cat,,
семейные пары,16490,0.079988
одинокие,4964,0.085012


### Вывод

Зависимость между семейным положением и возвратом кредита в срок есть, заемщики, состоящие в отношениях, чаще выплачивают кредит в срок, чем одинокие заемщики.
Заемщики в статусе женат / замужем выплачивают кредит в срок чаще, чем остальные.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
#Выявление процента должников по кредитам по отношению к доходу с разбивкой на категории

total_income_cat_purs = df.groupby('total_income_cat')['debt'].agg(['count','mean'])
total_income_cat_purs.sort_values('mean')


,count,mean
total_income_cat,,
Богатые,4270,0.070023
Нижний среднего класса,4463,0.079319
Обеспеченные,4084,0.082762
Cредний класс,4075,0.083681
Выше среднего класса,4562,0.089654


### Вывод

Заемщики с самым высоким доходом (от 215000) чаще всех выплачивают кредит в срок, а заемщики с доходом на класс ниже - реже всех.

- Как разные цели кредита влияют на его возврат в срок?

In [25]:
#Выявление процента должников по кредитам по отношению к целям кредита с разбивкой на категории

purpose_cat_purs = df.groupby('purpose_cat')['debt'].agg(['count','mean'])
purpose_cat_purs.sort_values('mean')



,count,mean
purpose_cat,,
Недвижимость,10811,0.072334
Свадьба,2324,0.080034
Образование,4013,0.092200
Автомобиль,4306,0.093590


In [26]:
#Отношение возврата кредита в срок по целям и количеству детей с помощью метода сводных таблиц
df.pivot_table(index='purpose_cat',
               columns='children_cat2',
               values=['debt'],
               aggfunc=['count','mean'])

count                mean          
                   debt                debt          
children_cat2 есть дети нет детей есть дети нет детей
purpose_cat                                          
Автомобиль         1461      2845  0.109514  0.085413
Недвижимость       3737      7074  0.081884  0.067289
Образование        1371      2642  0.102845  0.086677
Свадьба             794      1530  0.089421  0.075163

### Вывод

Заемщики, берущие кредит на надвижимость и свадьбы, возвращают его в срок чаще, чем те, кто берет кредит на образование или покупку автомобиля. Чаще всего возвращают в срок кредит на недвижимость заемщики, не имеющие детей. Реже всего возвращают в срок кредит на автомобиль заемщики, с детьми.

### Шаг 4. Общий вывод

Идеальный заемщик - это бездетный семьянин с высоким доходом, который берет кредит на покупку недвижимости.
Нежелательный заемщик - это одинокий человек с ребенком, чей доход является выше среднего (от 135000 до 165000) и который берёт кредит для покупки автомобиля.

В ходе работы:
 - 
определила и заполнила пропущенные значения
 - 
заменила вещественный тип данных на целочисленный
 - 
удалила дубликаты
 - 
выделила леммы в значениях столбца с целями получения кредита
 - 
категоризовала данные
